In [1]:
import re
import random
import numpy as np
import pandas as pd
import datetime
from faker import Faker


In [2]:
sales = pd.read_excel('data/sales_raw.xls', sheet_name = 'Orders')

sales.head()

,Row_ID,Order_Date,Customer_Name,City,Region,Category,Sub_Category,Product_Name,Sales,Quantity,Discount,Profit,Temperature,Is_Holiday,Fuel_Price
0,1,2016-11-08,Claire Gute,Henderson,South,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,42.31,False,2.572
1,2,2016-11-08,Claire Gute,Henderson,South,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,38.51,False,2.548
2,3,2016-06-12,Darrin Van Huff,Los Angeles,West,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,39.93,False,2.514
3,4,2015-10-11,Sean O'Donnell,Fort Lauderdale,South,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,46.63,True,2.561
4,5,2015-10-11,Sean O'Donnell,Fort Lauderdale,South,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,46.50,False,2.625


In [3]:
products = pd.read_csv('data/groceries_raw.csv')

products.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [4]:
sales.shape

(9994, 15)

In [5]:
products.shape

(38765, 3)

In [6]:
product_list = products.drop(
    columns = [
        'Member_number', 
        'Date'
    ]
).rename(
    columns = {
        'itemDescription': 'product'
    }
).drop_duplicates(
).reset_index(
    drop = True
)

In [7]:
product_list['product_id'] = product_list.reset_index()['index'].apply(
    lambda x: 'P' + f'{x + 1:04d}'
)

product_list['product'] = product_list['product'].apply(
    lambda x: x.title()
)

In [8]:
product_list.head()

,product,product_id
0,Tropical Fruit,P0001
1,Whole Milk,P0002
2,Pip Fruit,P0003
3,Other Vegetables,P0004
4,Rolls/Buns,P0005


In [9]:
len(product_list)

167

In [10]:
sales.drop(
    columns = [
        'Row_ID',
        'City',
        'Region',
        'Category',
        'Sub_Category',
        'Sales',
        'Discount',
        'Profit',
        'Temperature',
        'Is_Holiday',
        'Fuel_Price'
    ],
    inplace = True
)

sales.rename(
    columns = {
        'Order_Date': 'date_creation',
        'Customer_Name': 'name',
        'Product_Name': 'product_label',
        'Quantity': 'qty'
    },
    inplace = True
)

sales['name'] = sales['name'].apply(
    lambda x: x.replace('-', ' ')
)

In [11]:
sales.head()

,date_creation,name,product_label,qty
0,2016-11-08,Claire Gute,Bush Somerset Collection Bookcase,2
1,2016-11-08,Claire Gute,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",3
2,2016-06-12,Darrin Van Huff,Self-Adhesive Address Labels for Typewriters b...,2
3,2015-10-11,Sean O'Donnell,Bretford CR4500 Series Slim Rectangular Table,5
4,2015-10-11,Sean O'Donnell,Eldon Fold 'N Roll Cart System,2


In [12]:
customers = sales[['name']].drop_duplicates().reset_index(drop = True)

customers['ref_customer'] = customers.reset_index()['index'].apply(
    lambda x: 'C' + f'{x + 1:04d}'
)

In [13]:
sales = sales.merge(
    customers,
    how = 'inner',
    left_on = ['name'],
    right_on = ['name']
)

In [14]:
invoices = sales[['date_creation', 'ref_customer']].drop_duplicates().reset_index(drop = True)

invoices['id'] = invoices.reset_index()['index'].apply(
    lambda x: 'IV' + f'{x + 1:05d}'
)

In [15]:
invoices.head()

,date_creation,ref_customer,id
0,2016-11-08,C0001,IV00001
1,2017-01-26,C0001,IV00002
2,2015-10-15,C0001,IV00003
3,2016-06-12,C0002,IV00004
4,2017-12-04,C0002,IV00005


In [16]:
sales = sales.merge(
    invoices,
    how = 'inner',
    left_on = ['date_creation', 'ref_customer'],
    right_on = ['date_creation', 'ref_customer']
)

In [17]:
sales

,date_creation,name,product_label,qty,ref_customer,id
0,2016-11-08,Claire Gute,Bush Somerset Collection Bookcase,2,C0001,IV00001
1,2016-11-08,Claire Gute,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",3,C0001,IV00001
2,2017-01-26,Claire Gute,"SimpliFile Personal File, Black Granite, 15w x...",2,C0001,IV00002
3,2015-10-15,Claire Gute,C-Line Cubicle Keepers Polyproplyene Holder w/...,6,C0001,IV00003
4,2015-10-15,Claire Gute,Xerox 1986,1,C0001,IV00003
...,...,...,...,...,...,...
9989,2016-05-03,Susan MacKendrick,Tennsco Industrial Shelving,5,C0793,IV04992
9990,2016-05-03,Susan MacKendrick,"Computer Room Manger, 14""",2,C0793,IV04992
9991,2016-05-03,Susan MacKendrick,Logitech VX Revolution Cordless Laser Mouse fo...,3,C0793,IV04992
9992,2016-05-03,Susan MacKendrick,Cisco 8961 IP Phone Charcoal,3,C0793,IV04992


In [18]:
sales['firstname'] = sales['name'].apply(
    lambda x: x.split(" ", 1)[0]
)

sales['lastname'] = sales['name'].apply(
    lambda x: x.split(" ", 1)[1] if len(x.split(" ", 1)) > 1 else np.nan
)

In [19]:
sales.head()

,date_creation,name,product_label,qty,ref_customer,id,firstname,lastname
0,2016-11-08,Claire Gute,Bush Somerset Collection Bookcase,2,C0001,IV00001,Claire,Gute
1,2016-11-08,Claire Gute,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",3,C0001,IV00001,Claire,Gute
2,2017-01-26,Claire Gute,"SimpliFile Personal File, Black Granite, 15w x...",2,C0001,IV00002,Claire,Gute
3,2015-10-15,Claire Gute,C-Line Cubicle Keepers Polyproplyene Holder w/...,6,C0001,IV00003,Claire,Gute
4,2015-10-15,Claire Gute,Xerox 1986,1,C0001,IV00003,Claire,Gute


In [20]:
all_products = product_list[['product']].values.tolist()

for i in range(len(all_products)):
    all_products[i] = all_products[i][0]

In [21]:
sale_products = sales[['product_label']].drop_duplicates().reset_index(drop = True)

sale_products['product_mapping'] = sale_products['product_label'].apply(
    lambda x: random.choice(all_products)
)

In [22]:
sale_products.head()

,product_label,product_mapping
0,Bush Somerset Collection Bookcase,Fruit/Vegetable Juice
1,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Bottled Water
2,"SimpliFile Personal File, Black Granite, 15w x...",Fish
3,C-Line Cubicle Keepers Polyproplyene Holder w/...,Whole Milk
4,Xerox 1986,Bags


In [23]:
sales = sales.merge(
    sale_products,
    how = 'inner',
    left_on = ['product_label'],
    right_on = ['product_label']
).drop(
    columns = [
        'product_label'
    ]
).rename(
    columns = {
        'product_mapping': 'product_label'
    }
)

In [24]:
product_category = pd.read_csv('data/product_list_cat_map.csv')

product_category['type'] = product_category['type'].apply(
    lambda x: x.strip()
)

product_category.head()

,product,product_id,type
0,Tropical Fruit,P0001,Fruits&Vegetables
1,Whole Milk,P0002,Dairy
2,Pip Fruit,P0003,Fruits&Vegetables
3,Other Vegetables,P0004,Fruits&Vegetables
4,Rolls/Buns,P0005,Bakery


In [25]:
product_category.groupby('type')[['product']].count(
).reset_index(
).sort_values(
    by = 'product', 
    ascending = False
).reset_index(
    drop = True
)

,type,product
0,Dairy,20
1,Household items,20
2,Fruits&Vegetables,18
3,Meats,16
4,Snacks,15
5,Alcohol,12
6,Condiments,11
7,Dry/Baking Goods,11
8,Self-care,10
9,Beverages,9


In [26]:
category_price_range = pd.read_csv('data/category_price_range.csv')
category_price_range['max'] = category_price_range['max'].astype('float64')
category_price_range['min'] = category_price_range['min'].astype('str')
category_price_range['max'] = category_price_range['max'].astype('str')

category_price_range['range'] = category_price_range['min'] + ',' + category_price_range['max']

category_price_range.drop(
    columns = [
        'min',
        'max'
    ],
    inplace = True
)

In [27]:
product_category.head()

,product,product_id,type
0,Tropical Fruit,P0001,Fruits&Vegetables
1,Whole Milk,P0002,Dairy
2,Pip Fruit,P0003,Fruits&Vegetables
3,Other Vegetables,P0004,Fruits&Vegetables
4,Rolls/Buns,P0005,Bakery


In [28]:
product_category = product_category.merge(
    category_price_range,
    how = 'inner',
    left_on = ['type'],
    right_on = ['type']
)

In [29]:
product_category['subprice'] = product_category['range'].apply(
    lambda x: round(np.random.uniform(float(x.split(",", 1)[0]), float(x.split(",", 1)[1])), 2)
)

In [30]:
product_category.head()

,product,product_id,type,range,subprice
0,Tropical Fruit,P0001,Fruits&Vegetables,"2.0,6.0",2.05
1,Pip Fruit,P0003,Fruits&Vegetables,"2.0,6.0",4.53
2,Other Vegetables,P0004,Fruits&Vegetables,"2.0,6.0",5.45
3,Citrus Fruit,P0007,Fruits&Vegetables,"2.0,6.0",4.03
4,Packaged Fruit/Vegetables,P0013,Fruits&Vegetables,"2.0,6.0",4.47


In [31]:
sales = sales.merge(
    product_category,
    how = 'inner',
    left_on = ['product_label'],
    right_on = ['product']
).drop(
    columns = [
        'range'
    ]
).rename(
    columns = {
        'product': 'libelle'
    }
)

In [32]:
sales.head()

,date_creation,name,qty,ref_customer,id,firstname,lastname,product_label,libelle,product_id,type,subprice
0,2016-11-08,Claire Gute,2,C0001,IV00001,Claire,Gute,Fruit/Vegetable Juice,Fruit/Vegetable Juice,P0012,Beverages,1.27
1,2017-05-01,Arthur Gainer,3,C0152,IV01077,Arthur,Gainer,Fruit/Vegetable Juice,Fruit/Vegetable Juice,P0012,Beverages,1.27
2,2017-08-25,Seth Vernon,2,C0278,IV01976,Seth,Vernon,Fruit/Vegetable Juice,Fruit/Vegetable Juice,P0012,Beverages,1.27
3,2016-04-09,Herbert Flentye,5,C0521,IV03535,Herbert,Flentye,Fruit/Vegetable Juice,Fruit/Vegetable Juice,P0012,Beverages,1.27
4,2014-12-19,Irene Maddox,14,C0006,IV00029,Irene,Maddox,Fruit/Vegetable Juice,Fruit/Vegetable Juice,P0012,Beverages,1.27


In [33]:
sales['price'] = sales['subprice'] * sales['qty']

total_paid = sales.groupby('id')[['price']].sum().reset_index().rename(columns = {'price': 'totalpaid'})

In [34]:
sales = sales.merge(
    total_paid,
    how = 'inner',
    left_on = ['id'],
    right_on = ['id']
).sort_values(by = ['date_creation', 'id'])

In [35]:
sales.head()

,date_creation,name,qty,ref_customer,id,firstname,lastname,product_label,libelle,product_id,type,subprice,price,totalpaid
6703,2014-01-03,Darren Powers,2,C0025,IV00183,Darren,Powers,Tropical Fruit,Tropical Fruit,P0001,Fruits&Vegetables,2.05,4.10,4.10
1260,2014-01-04,Phillina Ober,2,C0286,IV02019,Phillina,Ober,Sweet Spreads,Sweet Spreads,P0121,Condiments,3.37,6.74,38.39
1261,2014-01-04,Phillina Ober,3,C0286,IV02019,Phillina,Ober,House Keeping Products,House Keeping Products,P0088,Household items,3.66,10.98,38.39
1262,2014-01-04,Phillina Ober,3,C0286,IV02019,Phillina,Ober,Frozen Dessert,Frozen Dessert,P0091,Snacks,6.89,20.67,38.39
3703,2014-01-05,Mick Brown,3,C0470,IV03212,Mick,Brown,Female Sanitary Products,Female Sanitary Products,P0084,Self-care,7.68,23.04,23.04


In [36]:
product_category = product_category.drop(
    columns = [
        'range'
    ]
).rename(
    columns = {
        'product_id': 'id',
        'product': 'label',
        'subprice': 'price'
    }
)

In [37]:
synth = Faker()

product_category['description'] = product_category['label'].apply(
    lambda x: synth.text(max_nb_chars = 50)
)

In [38]:
product_category['cost_price'] = product_category['price'].apply(
    lambda x: round(np.random.uniform((x / 2.5), (x / 3.5)), 2)
)

In [39]:
product_category['date_creation'] = product_category['label'].apply(
    lambda x: synth.date_between(datetime.date(2012, 1, 1), datetime.date(2012, 12, 31))
)

product_category['date_modification'] = product_category['date_creation'].apply(
    lambda x: x + pd.Timedelta(days = np.random.randint(30, 180))
)

In [40]:
product_category.head()

,label,id,type,price,description,cost_price,date_creation,date_modification
0,Tropical Fruit,P0001,Fruits&Vegetables,2.05,Right just can blood understand from place.,0.59,2012-07-21,2012-10-03
1,Pip Fruit,P0003,Fruits&Vegetables,4.53,Now energy position.,1.43,2012-01-16,2012-05-08
2,Other Vegetables,P0004,Fruits&Vegetables,5.45,Prevent hospital station public right at public.,1.76,2012-03-15,2012-05-20
3,Citrus Fruit,P0007,Fruits&Vegetables,4.03,Art floor sure after present plan.,1.48,2012-03-04,2012-08-11
4,Packaged Fruit/Vegetables,P0013,Fruits&Vegetables,4.47,Other parent wait look bring.,1.46,2012-06-15,2012-08-16
